---
# **High-Level Python ML Model to Synopsys ASIP Tools Comparable C**
### **Authors:** Cian O'Mahoney, Pedro Kreutz Werle, Ajay Kumar M
---

### Import the Required Packages:

In [ ]:
!pip install apache-tvm==0.14.dev273
!pip install tensorflow==2.12.0
!pip install keras==2.12.0
!pip install tflite==2.1.0

In [ ]:
import os
import pathlib
import numpy as np
import tarfile
from PIL import Image
from matplotlib import pyplot as plt
import shutil
import urllib.request

import tvm
import tvm.relay as relay
from tvm.relay.backend import Executor, Runtime
from tvm.micro import export_model_library_format

import tensorflow as tf

Folder name for all files generated by this notebook:

In [ ]:
FOLDER = "./py2c_VGG16"

### Download Images to Use as Input Data for Our Model:

In [ ]:
# Download datasets
os.makedirs(f"{FOLDER}/downloads")
os.makedirs(f"{FOLDER}/images")

urllib.request.urlretrieve(
    "https://data.deepai.org/stanfordcars.zip", f"{FOLDER}/downloads/target.zip"
)
urllib.request.urlretrieve(
    "http://images.cocodataset.org/zips/val2017.zip", f"{FOLDER}/downloads/random.zip"
)

# Extract them and rename their folders
shutil.unpack_archive(f"{FOLDER}/downloads/target.zip", f"{FOLDER}/downloads")
shutil.unpack_archive(f"{FOLDER}/downloads/random.zip", f"{FOLDER}/downloads")
shutil.move(f"{FOLDER}/downloads/cars_train/cars_train", f"{FOLDER}/images/target")
shutil.move(f"{FOLDER}/downloads/val2017", f"{FOLDER}/images/random")

### Rescale the Images to 64x64 and Normalise

In [ ]:
IMAGE_SIZE = (64, 64, 3)
unscaled_dataset = tf.keras.utils.image_dataset_from_directory(
    f"{FOLDER}/images",
    batch_size=32,
    shuffle=True,
    label_mode="categorical",
    image_size=IMAGE_SIZE[0:2],
)
rescale = tf.keras.layers.Rescaling(scale=1.0 / 255)
full_dataset = unscaled_dataset.map(lambda im, lbl: (rescale(im), lbl))

Display some sample images:

In [ ]:
num_target_class = len(os.listdir(f"{FOLDER}/images/target/"))
num_random_class = len(os.listdir(f"{FOLDER}/images/random/"))
print(f"{FOLDER}/images/target contains {num_target_class} images")
print(f"{FOLDER}/images/random contains {num_random_class} images")

# Show some samples and their labels
SAMPLES_TO_SHOW = 10
plt.figure(figsize=(20, 10))
for i, (image, label) in enumerate(unscaled_dataset.unbatch()):
    if i >= SAMPLES_TO_SHOW:
        break
    ax = plt.subplot(1, SAMPLES_TO_SHOW, i + 1)
    plt.imshow(image.numpy().astype("uint8"))
    plt.title(list(label.numpy()))
    plt.axis("off")

### Split the Dataset into Training and Testing:

In [ ]:
num_batches = len(full_dataset)
train_dataset = full_dataset.take(int(num_batches * 0.8))
validation_dataset = full_dataset.skip(len(train_dataset))

### Download Pretrained Model Weights:

In [ ]:
pretrained = tf.keras.applications.VGG16(
    input_shape=IMAGE_SIZE, weights='imagenet', include_top=False
)

### Modify Network to Fit Task:

In [ ]:
model = tf.keras.models.Sequential()

model.add(tf.keras.layers.InputLayer(input_shape=IMAGE_SIZE))
model.add(tf.keras.Model(inputs=pretrained.inputs, outputs=pretrained.layers[-5].output))

model.add(tf.keras.layers.Reshape((-1,)))
model.add(tf.keras.layers.Dropout(0.1))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(2, activation="softmax"))

### Train New Model on Training Data:

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005),
    loss="categorical_crossentropy",
    metrics=["accuracy"],
)
model.fit(train_dataset, validation_data=validation_dataset, epochs=30, verbose=2)

### Reduce Program Size by Quantising Model Parameters:

In [ ]:
def representative_dataset():
    for image_batch, label_batch in full_dataset.take(10):
        yield [image_batch]


converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8

quantized_model = converter.convert()

Download the Tensorflow model at this point

In [ ]:
directory_path = f"{FOLDER}/models"
os.makedirs(directory_path, exist_ok=True)
QUANTIZED_MODEL_PATH = f"{FOLDER}/models/quantized.tflite"
with open(QUANTIZED_MODEL_PATH, "wb") as f:
    f.write(quantized_model)

### Convert Tensorflow Model to C Source Code Using TVM:

In [ ]:
# Method to load model is different in TFLite 1 vs 2
try:  # TFLite 2.1 and above
    import tflite
    tflite_model = tflite.Model.GetRootAsModel(quantized_model, 0)
except AttributeError:  # Fall back to TFLite 1.14 method
    import tflite.Model
    tflite_model = tflite.Model.Model.GetRootAsModel(quantized_model, 0)

# Convert to the Relay intermediate representation
mod, params = tvm.relay.frontend.from_tflite(tflite_model)

# Set configuration flags to improve performance
target = tvm.target.Target("c")  # Specify target as generic C application.
runtime = tvm.relay.backend.Runtime("crt")  # Specify C runtime.
executor =  tvm.relay.backend.Executor("aot", {"unpacked-api": True, "interface-api": "c", "workspace-byte-alignment": 8})
config = {"tir.disable_vectorize": True}

# Convert to the MLF intermediate representation
with tvm.transform.PassContext(opt_level=3, config=config):
    mod = tvm.relay.build(mod, target, runtime=runtime, executor=executor, params=params)

parameter_size = len(tvm.runtime.save_param_dict(mod.get_params()))
print(f"Model parameter size: {parameter_size}")

### Download Model

In [ ]:
BUILD_DIR = pathlib.Path(FOLDER)
BUILD_DIR.mkdir(exist_ok=True)

# Now, we export the model into a tar file:
MODEL_NAME = "VGG16"
TAR_PATH = pathlib.Path(BUILD_DIR) / str(MODEL_NAME + ".tar")
export_model_library_format(mod, TAR_PATH)